# Actividad M3 - Traffic

El programa realiza una simulación del cruce de dos automóviles con un semáforo que determina el paso de cada carro

Autores:
**Rafael Alfonso Gomez Gonzalez**
Fecha de creación:
**22/Noviembre/2021**
Fecha de modificiación:
**23/Noviembre/2021**
**24/Noviembre/2021**
**25/Noviembre/2021**
**26/Noviembre/2021**
**27/Noviembre/2021**

In [491]:
import agentpy as ap
import matplotlib.pyplot as plt
import IPython
import random

In [492]:
# Type of move 
LEFT_RIGHT_MOVE = 1
UP_DOWN_MOVE = 3


VALID_MOVES = {
    LEFT_RIGHT_MOVE: (0, 1),
    UP_DOWN_MOVE: (1,0),
}

# Colors
CAR_COLOR = 1
YELLOW_COLOR = 101
GREEN_COLOR = 102
RED_COLOR = 103
ORANGE_COLOR = 104

# Set a ramdom seed
random.seed(9)

In [493]:
class CarAgent(ap.Agent):
    
    def setup(self):
        # return value of the type of move
        self.current_move_type = random.choice(list(VALID_MOVES.keys()))
        # On the start every car will move
        self.moving = True
        # Arrival time will start ar 0
        self.arrival_time = 0
    
    def get_move(self):
        # Returns the coordinate of the choosen move
        return VALID_MOVES[self.current_move_type]


In [494]:
class LightAgent(ap.Agent):
    def setup(self):
        # Iitilize the default color
        self.current_color = YELLOW_COLOR
        

In [495]:
class StreetModel(ap.Model):
    def setup(self):
        # Get positions in parameters
        first_position = self.p.cars_positions['first']
        second_position = self.p.cars_positions['second']

        # Convert to a tuple
        coordinates_position_one = [(first_position['Y'],first_position['X'])]
        coordinates_position_two = [(second_position['Y'],second_position['X'])]
        # Create an arary with the two available positions
        coordinates = coordinates_position_one + coordinates_position_two

        # Set the lights positions based on the given coordinates of cars
        x_light_position = first_position['Y']
        y_light_position = second_position['X']
        
        lights_location = [(x_light_position, y_light_position)]
        n_lights = len(lights_location)

        # Creation of traffic lights agent
        self.lights = ap.AgentList(self, n_lights, LightAgent)
        self.lights.type_agent = YELLOW_COLOR

        # Creation of cars agent
        self.cars = ap.AgentList(self, int(self.p["cars"]), CarAgent)
        self.cars.type_agent = CAR_COLOR
        self.cars.moving = True

        # Create grid (street)
        self.street = ap.Grid(self, [self.p["size"]["Y"], self.p["size"]["X"]], track_empty=True, torus=False)

        # Add the lights agents
        self.street.add_agents(self.lights, lights_location)
        # Convert every element in x_valid/y_valid to a tuple
        n_cars = self.p["cars"]
        valid_car_location = []
        while(n_cars):
            valid_car_location.append(random.choice(coordinates))
            n_cars-=1
        
        # Create an array of random locations previously created
        initial_car_location = [random.choice(valid_car_location) for _ in self.cars]
        for car, initial_location in zip(self.cars, initial_car_location):
            car.current_move_type = random.choice([LEFT_RIGHT_MOVE, UP_DOWN_MOVE])
        # Add cars
        self.street.add_agents(self.cars, initial_car_location)
    
    def step(self):
        # Check if a car is around traffic lights 
        for light in self.lights:
            posible_movement = []
            for neighbor in self.street.neighbors(light, distance= 1):
                # If the traffic light detects a car
                if neighbor.type_agent == CAR_COLOR:
                    neighbor.arrival_time = self.t
                    posible_movement.append(neighbor)
                    light.type_agent = RED_COLOR
            # If there's more than 1 car in the intersection
            if len(posible_movement) != 0:
                for car_ in posible_movement:
                    car_.moving = False
                posible_movement[0].moving=True
                posible_movement.pop(0)
            # Manage the case when the car is in the same spot as the traffic light
            for neighbor in self.street.neighbors(light, distance= 0):
                if neighbor.type_agent == CAR_COLOR:
                    neighbor.arrival_time +=1
                    neighbor.moving = True
                self.report((neighbor), neighbor.arrival_time/self.p['cars'])
            # When all the cars are gone the traffic light returns to the default state
            if len(posible_movement) == 0:
                light.type_agent = YELLOW_COLOR

        # Move every car if traffic light allows it
        for car in self.cars:
            coordinates_move = car.get_move()
            if car.moving == True:
                self.street.move_by(car, coordinates_move)
            else:
                pass
        
                    
    
    def end(self):
        pass

In [496]:
# Define parameters

parameters = {
    'cars':2,
    'cars_positions':{
        'first':{
            'X': 5,
            'Y': 10,
        },
        'second':{
            'X': 10,
            'Y': 5,
        }
    },
    'size': {'X': 20, 'Y': 20}, # Height and length of the grid
    'steps': 100,
}

In [497]:
def animation_plot(model, ax):
    attr_grid = model.street.attr_grid('type_agent')
    color_dict = {CAR_COLOR:'#0000FF', YELLOW_COLOR:'#FFFF00', GREEN_COLOR:'#00FF00', RED_COLOR:'#FF0000', ORANGE_COLOR: '#ff8400' ,None:'#0cc769'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Simulation of cars crossing an intersection\n"
                 f"Time-step: {model.t}")

fig, ax = plt.subplots()
model = StreetModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=5))

In [498]:
model.reporters 

{'seed': 90901135166841322720069264592089884270,
 CarAgent (Obj 2): 3.5,
 CarAgent (Obj 3): 4.0}